In [ ]:
"""

This is for morphological classification of galaxies by CNN,
New regresssion for B/D ratio
By Kenji Bekki, on 2018/3/30

"""


import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from keras.models import model_from_json
### Added 2018/3/30
from keras.applications import imagenet_utils
from keras.models import load_model
###
import keras.callbacks
import numpy as np
#import keras.backend.tensorflow_backend as KTF
#import tensorflow as tf
import os.path

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

### Total model number = (100*1) * nmodel

#iset=int(input('Input the total number of sets of models '))
#nmodel0=int(input('Input the total number of images per model '))
#nmodel=nmodel0*iset
#epochs=int(input('Input the number of epochs'))
#iset=5
#nmodel0=100
epochs=3000
#nmodel=nmodel0*iset
nmodel=1000
print('nmodel',nmodel)

### Original values
#batch_size = 128
#num_classes = 10
#epochs = 12
batch_size = 1
num_classes = 2
#epochs = 500
nb_epoch=epochs
n_mesh=50
#n_mesh=20
#nmodel=4000

img_rows, img_cols = n_mesh, n_mesh
n_mesh2=n_mesh*n_mesh-1
n_mesh3=n_mesh*n_mesh


print(img_rows, img_cols, n_mesh2)
#stop



#(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)

#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print(y_test.shape[0], 'y.test samples')
#print(str(y_test[0]))
#print(str(y_test[1]))
#print(str(y_test[2]))

#y_train = y_train.astype('int32')
#y_test = y_test.astype('int32')
#y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
#y_test =  keras.utils.np_utils.to_categorical(y_test, num_classes)

# This is for simlation data sets

with open('2dft.dat') as f:
  lines=f.readlines()
with open('2dftn1.dat') as f:
  lines1=f.readlines()
with open('2dftn2.dat') as f:
  lines2=f.readlines()


x_train=np.zeros((nmodel,n_mesh3))
x_test=np.zeros((nmodel,n_mesh3))
#y_train=np.zeros(nmodel,dtype=np.int)
#y_test=np.zeros(nmodel,dtype=np.int)
y_train=np.zeros((nmodel,2))
y_test=np.zeros((nmodel,2))
#y_test=np.zeros(nmodel)
#print(y_train)

# For 2D density map data
ibin=0
jbin=-1
for num,j in enumerate(lines):
  jbin=jbin+1
  tm=j.strip().split()
  x_train[ibin,jbin]=float(tm[0])
  x_test[ibin,jbin]=float(tm[0])
#  print('ibin,jbin',ibin,jbin)
  if jbin == n_mesh2:
    ibin+=1
    jbin=-1

# For morphological map (theta)
ibin=0
for num,j in enumerate(lines1):
  tm=j.strip().split()
  y_train[ibin,0]=float(tm[0])
  y_test[ibin,0]=float(tm[0])
#  y_train[ibin]=int(tm[0])-1
#  y_test[ibin]=int(tm[0])-1
#  print('ibin, (Morpholigcl type)',ibin,y_train[ibin])
  ibin+=1

# For morphological map (phi)
ibin=0
for num,j in enumerate(lines2):
  tm=j.strip().split()
  y_train[ibin,1]=float(tm[0])
  y_test[ibin,1]=float(tm[0])
#  y_train[ibin]=int(tm[0])-1
#  y_test[ibin]=int(tm[0])-1
#  print('ibin, (Morpholigcl type)',ibin,y_train[ibin])
  ibin+=1



x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
# For laelling
#y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
#y_test =  keras.utils.np_utils.to_categorical(y_test, num_classes)

print('Galaxy type',y_train[:5])

#stop

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

### For labelling of morphological types

#model.add(Dense(num_classes, activation='softmax'))
#model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])

### For regression of B/D

model.add(Dense(2, activation='linear'))
#model.add(Dense(1))
#model.add(Activation=('linear'))
#model.add(activation=('linear'))
model.compile(loss='mean_squared_error',
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


print('save the architecture of a model')

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")






nmodel 1000
50 50 2499
Galaxy type [[4.15999413 3.1924963 ]
 [4.15999413 3.1924963 ]
 [4.15999413 3.1924963 ]
 [4.15999413 3.1924963 ]
 [4.15999413 3.1924963 ]]
Epoch 1/3000
1000/1000 [==============================] - 11s 8ms/step - loss: 30.0841 - accuracy: 0.4899 - val_loss: 20.7619 - val_accuracy: 0.5310
Epoch 2/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 19.4404 - accuracy: 0.5472 - val_loss: 11.0989 - val_accuracy: 0.5300
Epoch 3/3000
1000/1000 [==============================] - 8s 8ms/step - loss: 11.2384 - accuracy: 0.4676 - val_loss: 8.3564 - val_accuracy: 0.5300
Epoch 4/3000
1000/1000 [==============================] - 8s 8ms/step - loss: 9.3619 - accuracy: 0.5325 - val_loss: 8.0226 - val_accuracy: 0.5300
Epoch 5/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.7597 - accuracy: 0.4925 - val_loss: 8.0036 - val_accuracy: 0.5300
Epoch 6/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.3632 - accuracy: 0.5424

1000/1000 [==============================] - 7s 7ms/step - loss: 8.6197 - accuracy: 0.5320 - val_loss: 7.9064 - val_accuracy: 0.5640
Epoch 56/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.6629 - accuracy: 0.5210 - val_loss: 7.8939 - val_accuracy: 0.5400
Epoch 57/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.6272 - accuracy: 0.4768 - val_loss: 7.8989 - val_accuracy: 0.5300
Epoch 58/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.8588 - accuracy: 0.4812 - val_loss: 7.8972 - val_accuracy: 0.5600
Epoch 59/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.7765 - accuracy: 0.5213 - val_loss: 7.8883 - val_accuracy: 0.5300
Epoch 60/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.5494 - accuracy: 0.5041 - val_loss: 7.8956 - val_accuracy: 0.5300
Epoch 61/3000
1000/1000 [==============================] - 8s 8ms/step - loss: 9.1629 - accuracy: 0.5037 - val_loss: 7.8730 - val_ac

Epoch 111/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.1840 - accuracy: 0.5276 - val_loss: 7.7892 - val_accuracy: 0.5460
Epoch 112/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.5280 - accuracy: 0.5137 - val_loss: 7.7654 - val_accuracy: 0.5750
Epoch 113/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.1784 - accuracy: 0.5184 - val_loss: 7.7649 - val_accuracy: 0.5550
Epoch 114/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.4896 - accuracy: 0.5313 - val_loss: 7.7755 - val_accuracy: 0.5600
Epoch 115/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.8291 - accuracy: 0.5112 - val_loss: 7.7835 - val_accuracy: 0.6530
Epoch 116/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.6582 - accuracy: 0.5296 - val_loss: 7.7842 - val_accuracy: 0.7140
Epoch 117/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.3427 - accuracy: 0.5553 - val_

1000/1000 [==============================] - 7s 7ms/step - loss: 8.4537 - accuracy: 0.5226 - val_loss: 7.6715 - val_accuracy: 0.6230
Epoch 167/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.8258 - accuracy: 0.5117 - val_loss: 7.6844 - val_accuracy: 0.6100
Epoch 168/3000
1000/1000 [==============================] - 8s 8ms/step - loss: 8.4341 - accuracy: 0.5355 - val_loss: 7.6628 - val_accuracy: 0.6400
Epoch 169/3000
1000/1000 [==============================] - 9s 9ms/step - loss: 8.8696 - accuracy: 0.5655 - val_loss: 7.6379 - val_accuracy: 0.6720
Epoch 170/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.3085 - accuracy: 0.5731 - val_loss: 7.6321 - val_accuracy: 0.6160
Epoch 171/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.1840 - accuracy: 0.5138 - val_loss: 7.6341 - val_accuracy: 0.6400
Epoch 172/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.2377 - accuracy: 0.5393 - val_loss: 7.6399 - 

1000/1000 [==============================] - 7s 7ms/step - loss: 8.1512 - accuracy: 0.5388 - val_loss: 7.5021 - val_accuracy: 0.7260
Epoch 222/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.1544 - accuracy: 0.5728 - val_loss: 7.4679 - val_accuracy: 0.7260
Epoch 223/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 7.8585 - accuracy: 0.5983 - val_loss: 7.4735 - val_accuracy: 0.6970
Epoch 224/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 7.8854 - accuracy: 0.5931 - val_loss: 7.4729 - val_accuracy: 0.7220
Epoch 225/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.2511 - accuracy: 0.5351 - val_loss: 7.4698 - val_accuracy: 0.7260
Epoch 226/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.2037 - accuracy: 0.5475 - val_loss: 7.4537 - val_accuracy: 0.6160
Epoch 227/3000
1000/1000 [==============================] - 7s 7ms/step - loss: 8.1884 - accuracy: 0.5489 - val_loss: 7.4446 - 